In [8]:
import cv2
import mediapipe as mp
import csv
import os
import numpy as np
from mediapipe.framework.formats import landmark_pb2

# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

def read_keypoints_from_csv(csv_file):
    keypoints = []
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        header = next(reader)
        for row in reader:
            frame_keypoints = []
            for i in range(33):
                x = float(row[1 + i*4])
                y = float(row[2 + i*4])
                z = float(row[3 + i*4])
                visibility = float(row[4 + i*4])
                frame_keypoints.append(landmark_pb2.NormalizedLandmark(x=x, y=y, z=z, visibility=visibility))
            keypoints.append(frame_keypoints)
    return keypoints

def draw_keypoints_on_video(video_path, keypoints, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}.")
        return
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Ensure the output directory exists
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_count < len(keypoints):
            frame_keypoints = keypoints[frame_count]
            pose_landmarks = landmark_pb2.NormalizedLandmarkList(landmark=frame_keypoints)
            mp_drawing.draw_landmarks(frame, pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        out.write(frame)
        frame_count += 1
    
    cap.release()
    out.release()
    print(f"Processed {frame_count} frames. Output saved to {output_path}")

# 사용 예시
csv_file1 = 'C:/Users/jk/action_assess_2/data/csv/Z76/313-2-1-15-Z76_D.csv'
csv_file2 = 'C:/Users/jk/action_assess_2/data/csv/Z106/313-2-1-15-Z106_D.csv'
video_path1 = 'C:/Users/jk/action_assess_2/data/video/Z76/313-2-1-15-Z76_D.avi'
video_path2 = 'C:/Users/jk/action_assess_2/data/video/Z106/313-2-1-15-Z106_D.avi'
output_path1 = 'C:/Users/jk/action_assess_2/data/video/Z76_keypoints/keypoints_313-2-1-15-Z76_D.avi'
output_path2 = 'C:/Users/jk/action_assess_2/data/video/Z106_keypoints/keypoints_313-2-1-15-Z106_D.avi'

keypoints1 = read_keypoints_from_csv(csv_file1)
keypoints2 = read_keypoints_from_csv(csv_file2)

draw_keypoints_on_video(video_path1, keypoints1, output_path1)
draw_keypoints_on_video(video_path2, keypoints2, output_path2)

Processed 32 frames. Output saved to C:/Users/jk/action_assess_2/data/video/Z76_keypoints/keypoints_313-2-1-15-Z76_D.avi
Processed 32 frames. Output saved to C:/Users/jk/action_assess_2/data/video/Z106_keypoints/keypoints_313-2-1-15-Z106_D.avi
